In [1]:
import os
import json
import cv2
import pandas as pd
from tqdm import tqdm
import numpy as np


In [2]:
os.listdir()

['fixed_gan.ipynb',
 'simple_model.pth',
 '.DS_Store',
 'models.py',
 'measures_categories.csv',
 'score_category.ipynb',
 'icon_legend.json',
 'design_topics.csv',
 'measures_context.csv',
 '__pycache__',
 '10594-screenshot.jpg',
 'component_legend.json',
 'README.md',
 'measures.csv',
 '.gitignore',
 '10594-hierarchy.json',
 'simple_model.pt',
 'MakeDataset.ipynb',
 'textButton_legend.json',
 'gflow_net.ipynb',
 'combined',
 '.git',
 '10594-wireframe.png',
 '10594-metadata.json',
 'model.ipynb',
 'model_clip.ipynb',
 'semantic_annotations']

1. Density - Anushka

In [3]:
def calculate_area(bounds):
    return (bounds[2] - bounds[0]) * (bounds[3] - bounds[1])

def do_rectangles_overlap(rect1, rect2):
    return not (rect1[2] <= rect2[0] or rect1[0] >= rect2[2] or rect1[3] <= rect2[1] or rect1[1] >= rect2[3])

def merge_overlapping_objects(data):
    merged_objects = []

    if 'children' in data:
        for parent in data['children']:
            non_overlapping_children = []

            if 'children' in parent:
                for child in parent['children']:
                    overlapping = False
                    for merged_obj in merged_objects:
                        if do_rectangles_overlap(child['bounds'], merged_obj['bounds']):
                            overlapping = True
                            break

                    if not overlapping:
                        non_overlapping_children.append(child)

                merged_objects.extend(non_overlapping_children)

    return merged_objects

def calculate_density_measure(total_area, frame_area):
    if total_area > 0 and frame_area > 0:
        density_measure = 1 - 2 * abs(0.5 - total_area / frame_area)
        density_measure = max(0, min(1, density_measure))
    else:
        density_measure = 0

    return density_measure

def calculate_density(data):
    total_area = 0

    merged_objects = merge_overlapping_objects(data)

    for item in merged_objects:
        bounds = item.get('bounds')
        if bounds:
            area = calculate_area(bounds)
            total_area += area

    frame_bounds = data.get('bounds')
    if frame_bounds:
        frame_area = calculate_area(frame_bounds)

    density_measure = calculate_density_measure(total_area, frame_area)
    scaled_density_measure = 1 - (abs(density_measure - 0.5) * 2)

    return scaled_density_measure

2. Colour - Yash

In [4]:
def calculate_colorfulness(image):
    image = cv2.imread(image)
    # Convert the image to sRGB color space
    srgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Calculate the pixel cloud along directions (rg, yb)
    rg = srgb_image[:,:,0] - srgb_image[:,:,1]
    yb = (srgb_image[:,:,0] + srgb_image[:,:,1]) / 2 - srgb_image[:,:,2]

    # Calculate the standard deviation and mean value along directions (rg, yb)
    std_rg = np.std(rg)
    std_yb = np.std(yb)
    mean_rg = np.mean(rg)
    mean_yb = np.mean(yb)

    # Calculate ^M(3) colorfulness metric
    colorfulness = np.sqrt(std_rg**2 + std_yb**2) + 0.3 * np.sqrt(mean_rg**2 + mean_yb**2)
    #make colour between 0 and 1
    # colorfulness = colorfulness / 200

    return colorfulness

3. Proportion - Devesh

In [5]:
# 1640
# 39266
standard_proportions = {
        'sq': 1,        # Square
        'r2': 1/1.414,  # Square root of 2
        'gr': 1/1.618,  # Golden ratio
        'r3': 1/1.732,  # Square root of 3
        'ds': 1/2       # Double square
    }

# Recursive function to parse JSON and calculate PM_object
def parse_json_and_calculate_PMobject(component, standard_proportions):
    sum_min_diff = 0
    n = 0

    def extract_proportion(bounds):
        x1, y1, x2, y2 = bounds
        width = x2 - x1
        height = y2 - y1
        r = height / width if width != 0 else 0
        if width != 0 and r <= 1:
            return r
        elif width != 0 and r > 1:
            return 1/r
        else:
            return r

    # Recursively process components and children
    def process_component(component):
        nonlocal sum_min_diff, n
        bounds = component.get("bounds", [])
        if bounds:
            proportion = extract_proportion(bounds)
            min_diff = min(abs(proportion - sp) for sp in standard_proportions.values())
            sum_min_diff += (1 - min_diff / 0.5)
            n += 1
        for child in component.get("children", []):
            process_component(child)

    process_component(component)
    return abs(sum_min_diff / n) if n != 0 else 0

# Calculate the PM_object for the entire JSON data structure
# PM_object = parse_json_and_calculate_PMobject(json_data, standard_proportions)
# PM_object

def calculate_PMlayout(width_layout, height_layout):
    # Standard proportions as per the provided screenshot

    # Calculate the layout proportion
    r_layout = height_layout / width_layout if width_layout != 0 else 0
    p_layout = r_layout if r_layout <= 1 else 1 / r_layout

    # Find the minimum difference between p_layout and the standard proportions
    min_diff = min(abs(p_layout - sp) for sp in standard_proportions.values())

    # Calculate PM_layout according to the formula
    PM_layout = 1 - (min_diff / 0.5)

    return abs(PM_layout)

# calculate_PMlayout(json_data["bounds"][2],json_data["bounds"][3])

def calculate_PM(json_data):
    pm_object = parse_json_and_calculate_PMobject(json_data, standard_proportions)
    pm_layout = calculate_PMlayout(json_data["bounds"][2],json_data["bounds"][3])
    return (pm_object + pm_layout) / 2

In [6]:
# with open("semantic_annotations/39266.json") as f:
#     data = json.load(f)
    
# calculate_PM(data)

4. Symmetry - Mann

In [7]:
import os
import json
import math

def determine_quadrant(element_center_x, element_center_y, center_x, center_y):
    if element_center_x < center_x and element_center_y < center_y:
        return 'UL'  # Upper-Left
    elif element_center_x >= center_x and element_center_y < center_y:
        return 'UR'  # Upper-Right
    elif element_center_x < center_x and element_center_y >= center_y:
        return 'LL'  # Lower-Left
    elif element_center_x >= center_x and element_center_y >= center_y:
        return 'LR'  # Lower-Right

def calculate_element_properties(bounds):
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    center_x = bounds[0] + width / 2
    center_y = bounds[1] + height / 2
    return center_x, center_y, width, height

def extract_ui_elements(data, parent_bounds=None):
    elements = []
    bounds = data.get('bounds', parent_bounds)
    if 'children' not in data or not data['children']:
        return [{'class': data['class'], 'bounds': bounds}]
    for child in data['children']:
        elements.extend(extract_ui_elements(child, bounds))
    return elements

# Function to calculate the symmetry score of a single UI screen
def calculate_symmetry(data):
    screen_bounds = data['bounds']
    screen_center_x = (screen_bounds[2] + screen_bounds[0]) / 2
    screen_center_y = (screen_bounds[3] + screen_bounds[1]) / 2

    # Extract UI elements
    ui_elements = extract_ui_elements(data)

    # Organize elements by quadrant
    quadrants = {'UL': [], 'UR': [], 'LL': [], 'LR': []}
    for element in ui_elements:
        center_x, center_y, width, height = calculate_element_properties(element['bounds'])
        quadrant = determine_quadrant(center_x, center_y, screen_center_x, screen_center_y)
        quadrants[quadrant].append({
            'center_x': center_x,
            'center_y': center_y,
            'width': width,
            'height': height
        })

    # Calculate averages for each quadrant
    quadrant_sums = {key: {'x_sum': 0, 'y_sum': 0, 'width_sum': 0, 'height_sum': 0, 'count': 0}
                     for key in quadrants.keys()}

    for quadrant, elements in quadrants.items():
        for element in elements:
            quadrant_sums[quadrant]['x_sum'] += element['center_x']
            quadrant_sums[quadrant]['y_sum'] += element['center_y']
            quadrant_sums[quadrant]['width_sum'] += element['width']
            quadrant_sums[quadrant]['height_sum'] += element['height']
            quadrant_sums[quadrant]['count'] += 1

    quadrant_averages = {}
    for quadrant, sums in quadrant_sums.items():
        count = sums['count']
        if count > 0:
            quadrant_averages[quadrant] = {
                'avg_center_x': sums['x_sum'] / count,
                'avg_center_y': sums['y_sum'] / count,
                'avg_width': sums['width_sum'] / count,
                'avg_height': sums['height_sum'] / count
            }
        else:
            quadrant_averages[quadrant] = None

    # Calculate symmetry values for vertical, horizontal
    symmetry_values = {'SYM_vertical': 0, 'SYM_horizontal': 0, 'SYM_radial': 0}
    if quadrant_averages['UL'] and quadrant_averages['UR']:
        symmetry_values['SYM_vertical'] = 1 - (
            abs(quadrant_averages['UL']['avg_center_x'] - quadrant_averages['UR']['avg_center_x']) /
            screen_center_x
        )

    if quadrant_averages['UL'] and quadrant_averages['LR']:
        symmetry_values['SYM_horizontal'] = 1 - (
            abs(quadrant_averages['UL']['avg_center_y'] - quadrant_averages['LR']['avg_center_y']) /
            screen_center_y
        )

    # Since we might not have elements in all quadrants, use available symmetry values
    overall_symmetry = 0
    symmetry_count = 0
    for symmetry in symmetry_values.values():
        if symmetry > 0:
            overall_symmetry += symmetry
            symmetry_count += 1

    if symmetry_count > 0:
        overall_symmetry /= symmetry_count  # Average symmetry score

    return overall_symmetry

5. Balance - Vedika

In [8]:
import json

def screen_bounds(json_data):
    bounds = json_data['bounds']
    left, top, right, bottom = bounds
    width = right - left
    height = bottom - top
    return width, height

def parse_annotations(json_data):
    objects = []
    
    def parse_children(children):
        for item in children:
            if 'bounds' in item:
                left, top, right, bottom = item['bounds']
                width = right - left
                height = bottom - top
                objects.append({
                    'left': left,
                    'top': top,
                    'right': right,
                    'bottom': bottom,
                    'width': width,
                    'height': height
                })
            if 'children' in item:
                parse_children(item['children'])

    parse_children(json_data['children'])
    return objects

def compute_balance_scores(objects, json_data):
    screen_width, screen_height = screen_bounds(json_data)
    left_area = right_area = top_area = bottom_area = 0
    left_distance = right_distance = top_distance = bottom_distance = 0
    
    if objects == []:
        return 0
    else:
        for obj in objects:
            if (obj['left'] + obj['right']) / 2 < screen_width / 2:
                left_area += obj['width'] * obj['height']
                left_distance += abs((obj['left'] + obj['width']) / 2 - screen_width)
            else:
                right_area += obj['width'] * obj['height']
                right_distance += abs((obj['left'] + obj['width']) / 2 - screen_width)
            
            if (obj['top'] + obj['bottom']) / 2 < screen_height / 2:
                top_area += obj['width'] * obj['height']
                top_distance += abs((obj['top'] + obj['height']) / 2 - screen_height)
            else:
                bottom_area += obj['width'] * obj['height']
                bottom_distance += abs((obj['top'] + obj['height']) / 2 - screen_height)
      
        left_weight = left_area / max(left_area, right_area) if max(left_area, right_area) != 0 else 0
        right_weight = right_area / max(left_area, right_area) if max(left_area, right_area) != 0 else 0
        vertical_balance = abs(left_weight - right_weight)

        top_weight = top_area / max(top_area, bottom_area) if max(top_area, bottom_area) != 0 else 0
        bottom_weight = bottom_area / max(top_area, bottom_area) if max(top_area, bottom_area) != 0 else 0
        horizontal_balance = abs(top_weight - bottom_weight)
      
        balance_measure = abs((vertical_balance + horizontal_balance) / 2)
    
    return balance_measure

def balance_score(json_data):
    objects = parse_annotations(json_data)
    balance_score = compute_balance_scores(objects, json_data)
    # print("Balance Score:", balance_score)
    return balance_score

**Calculate Everything**

In [9]:
# import zipfile
# import os

# # Assuming we have a path to the zip file and a target directory
# zip_file_path = './ui_layout_vectors.zip'  # Replace with your zip file path
# target_directory = './'  # Replace with your target directory

# # Create target directory if it does not exist
# if not os.path.exists(target_directory):
#     os.makedirs(target_directory)

# # Extract the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(target_directory)

# # The code above assumes the paths are known and correctly provided.
# # If you have the zip file already in your environment, you can adjust the paths accordingly.


In [10]:
topics = pd.read_csv("design_topics.csv")
mapping_ids = []
for index,i in topics.iterrows():
    mapping_ids.append(i.values)

In [11]:
output_csv = './measures.csv'
json_folder = './semantic_annotations'
image_folder = './combined'

page_names = []
balance_measures = []
colour_measures = []
symmetry_measures = []
proportion_measures = []
density_measures = []

progress_bar = tqdm(total=len(os.listdir(json_folder)), desc="Processing JSON files")

for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        progress_bar.update(1)  # Update progress bar
        page_number = filename.split('.')[0]
        page_names.append(page_number)

        with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as f:
            data = json.load(f)

        balance = balance_score(data)
        balance_measures.append(balance)

        proportion = calculate_PM(data)
        proportion_measures.append(proportion)

        symmetry = calculate_symmetry(data)
        symmetry_measures.append(symmetry)

        density = calculate_density(data)
        density_measures.append(density)

        # Load corresponding image for colour calculation
        image_filename = page_number + '.jpg'
        image_path = os.path.join(image_folder, image_filename)
        # print(image_path)
        if os.path.exists(image_path):
            colour = calculate_colorfulness(image_path)
            colour_measures.append(colour)
            # print(f"Color value for {page_number} : {colour}")
        else:
            # print("no file exists")
            colour_measures.append(None)  # Handle case where image is not found
        # print("------")

        # Log progress every 1000 pages
        if len(page_names) % 2000 == 0:
            print(f"Processed {len(page_names)} pages")

progress_bar.close()

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Page': page_names,
    'Balance': balance_measures,
    'Colour': colour_measures,
    'Symmetry': symmetry_measures,
    'Proportion': proportion_measures,
    'Density': density_measures
})

# Calculate final score (average of all measures)
results_df['Colour'] = results_df['Colour'] / 200 # to scale between 0-1
results_df['Final Score'] = results_df[['Balance', 'Colour', 'Symmetry', 'Proportion', 'Density']].mean(axis=1)
# results_df['Final Score'] = results_df[['Balance', 'Symmetry', 'Proportion', 'Density']].mean(axis=1)

# Filtering outliers (dropping 1 out of 300 entries, negligible)
results_df = results_df[results_df['Final Score'] <= 1] 

# Save results to CSV
results_df.to_csv(output_csv, index=False)

print("Results saved to", output_csv)

Processing JSON files:   2%|▏         | 2010/132524 [00:26<27:34, 78.87it/s]

Processed 2000 pages


Processing JSON files:   3%|▎         | 4012/132524 [00:53<29:06, 73.57it/s]

Processed 4000 pages


Processing JSON files:   5%|▍         | 6015/132524 [01:19<28:46, 73.28it/s]

...

Processed 64000 pages


Processing JSON files:  50%|████▉     | 66009/132524 [14:51<14:12, 78.06it/s]

Processed 66000 pages


Processing JSON files:  50%|████▉     | 66261/132524 [14:54<14:54, 74.06it/s]


Results saved to ./measures.csv


In [12]:
output_csv = './measures_context.csv'
json_folder = './semantic_annotations'
image_folder = './combined'

page_names = []
balance_measures = []
colour_measures = []
symmetry_measures = []
proportion_measures = []
density_measures = []
descriptions = []

progress_bar = tqdm(total=len(os.listdir(json_folder)), desc="Processing JSON files")

for c,i in enumerate(mapping_ids):
    filename = f"{i[0]}.json"
    if filename.endswith('.json'):
        progress_bar.update(1)  # Update progress bar

        if os.path.isfile(os.path.join(json_folder, filename)):
            with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as f:
                data = json.load(f)
        else:
            continue
        
        page_number = filename.split('.')[0]
        page_names.append(page_number)

        balance = balance_score(data)
        balance_measures.append(balance)

        proportion = calculate_PM(data)
        proportion_measures.append(proportion)

        symmetry = calculate_symmetry(data)
        symmetry_measures.append(symmetry)

        density = calculate_density(data)
        density_measures.append(density)
        
        descriptions.append(i[1])

        # Load corresponding image for colour calculation
        image_filename = page_number + '.jpg'
        image_path = os.path.join(image_folder, image_filename)
        # print(image_path)
        if os.path.exists(image_path):
            colour = calculate_colorfulness(image_path)
            colour_measures.append(colour)
            # print(f"Color value for {page_number} : {colour}")
        else:
            # print("no file exists")
            colour_measures.append(None)  # Handle case where image is not found
        # print("------")

        # Log progress every 1000 pages
        if len(page_names) % 2000 == 0:
            print(f"Processed {len(page_names)} pages")

progress_bar.close()

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Page': page_names,
    'Balance': balance_measures,
    'Colour': colour_measures,
    'Symmetry': symmetry_measures,
    'Proportion': proportion_measures,
    'Density': density_measures,
    'Description': descriptions
})

# Calculate final score (average of all measures)
results_df['Colour'] = results_df['Colour'] / 200 # to scale between 0-1
results_df['Final Score'] = results_df[['Balance', 'Colour', 'Symmetry', 'Proportion', 'Density']].mean(axis=1)
# results_df['Final Score'] = results_df[['Balance', 'Symmetry', 'Proportion', 'Density']].mean(axis=1)

# Filtering outliers (dropping 1 out of 300 entries, negligible)
results_df = results_df[results_df['Final Score'] <= 1] 

# Save results to CSV
results_df.to_csv(output_csv, index=False)

print("Results saved to", output_csv)

Processing JSON files:   1%|          | 1460/132524 [00:17<26:48, 81.46it/s]

Results saved to ./measures_context.csv


In [13]:
df = pd.read_csv("measures.csv")
df.describe()

,Page,Balance,Colour,Symmetry,Proportion,Density,Final Score
count,66249.000000,66249.000000,66249.000000,66249.000000,66249.000000,66249.000000,66249.000000
mean,36072.580703,0.695085,0.598781,0.138545,0.810199,0.301142,0.508750
std,20904.422841,0.243705,0.175356,0.194888,0.084053,0.302196,0.085954
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007424
25%,17856.000000,0.519113,0.487014,0.000000,0.757232,0.000000,0.452564
50%,36178.000000,0.734394,0.599987,0.051181,0.808285,0.213646,0.507913
75%,54241.000000,0.908731,0.730558,0.214001,0.859998,0.526863,0.562342
max,72218.000000,1.732173,1.262280,0.999582,2.327174,0.999996,0.905646


In [18]:
df[df["Balance"] > 1]

,Page,Balance,Colour,Symmetry,Proportion,Density,Final Score
235,14518,1.108263,0.528254,0.285818,0.820340,0.419411,0.632417
3087,4748,1.330486,0.688026,0.000000,0.610342,0.212674,0.568306
10391,49190,1.162551,0.637811,0.058737,0.771291,0.000000,0.526078
11365,34818,1.191943,0.390435,0.011806,0.550000,0.314863,0.491809
24507,14070,1.116827,0.566883,0.000000,0.730774,0.135922,0.510081
28526,56445,1.293240,0.700323,0.136574,0.750429,0.000000,0.576113
29292,53775,1.272726,0.860029,0.302083,0.573512,0.000000,0.601670
29752,8849,1.732173,0.536671,0.203935,0.590874,0.000000,0.612731
36859,8165,1.692374,0.555232,0.000000,0.498826,0.000000,0.549286
42832,52015,1.170661,0.376859,0.062695,0.590300,0.442768,0.528657


In [17]:
df[df["Proportion"] > 1]

,Page,Balance,Colour,Symmetry,Proportion,Density,Final Score
6486,1511,0.564834,0.775499,0.068056,1.113131,0.292642,0.562832
8186,38816,0.608375,0.655269,0.266204,1.723629,0.162078,0.683111
8418,38817,0.440054,0.670884,0.051522,2.321343,0.761134,0.848987
12979,40554,0.429681,0.619110,0.374640,1.623590,0.590204,0.727445
30057,52349,0.565720,0.601457,0.000000,2.140121,0.323359,0.726132
30866,60900,0.565720,0.602002,0.000000,2.140121,0.323359,0.726241
39574,20686,0.859946,0.821130,0.390278,1.212113,0.836083,0.823910
49503,38813,0.765944,0.671781,0.053333,1.858556,0.721863,0.814295
52701,38822,0.439952,0.630182,0.047633,2.327174,0.766546,0.842297
56149,2981,1.018943,0.839108,0.260097,1.231169,0.447140,0.759291


In [21]:
df[df["Colour"] > 1].count()

Page           10
Balance        10
Colour         10
Symmetry       10
Proportion     10
Density        10
Final Score    10
dtype: int64